<a href="https://colab.research.google.com/github/ranjith13119/Seq2Seq---Text-Summarization/blob/main/WROD2VEC_SENTIMENT_ANALYSYS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://machinelearningmastery.com/use-word-embedding-layers-deep-learning-keras/

https://github.com/codebasics/deep-learning-keras-tf-tutorial

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
docs = ['Well done!',
		'Good work',
		'Great effort',
		'nice work',
		'Excellent!',
		'Weak',
		'Poor effort!',
		'not good',
		'poor work',
		'Could have done better.']
# define class labels
labels = np.array([1,1,1,1,1,0,0,0,0,0])

In [ ]:
VOCAB_SIZE = 50
encoded_docs = [one_hot(d, VOCAB_SIZE) for d in docs]  # onhto representation

In [ ]:
encoded_docs

[[38, 30],
 [48, 17],
 [40, 31],
 [7, 17],
 [25],
 [30],
 [7, 31],
 [20, 48],
 [7, 17],
 [11, 23, 30, 7]]

In [ ]:
max_length = 4
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

In [ ]:
padded_docs

array([[38, 30,  0,  0],
       [48, 17,  0,  0],
       [40, 31,  0,  0],
       [ 7, 17,  0,  0],
       [25,  0,  0,  0],
       [30,  0,  0,  0],
       [ 7, 31,  0,  0],
       [20, 48,  0,  0],
       [ 7, 17,  0,  0],
       [11, 23, 30,  7]], dtype=int32)

Binary classification model

In [ ]:
model = Sequential()
model.add(Embedding(VOCAB_SIZE, 8, input_length=max_length))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
print(model.summary())
# fit the model
model.fit(padded_docs, labels, epochs=50, verbose=0)
# evaluate the model
loss, accuracy = model.evaluate(padded_docs, labels, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 4, 8)              400       
_________________________________________________________________
flatten_2 (Flatten)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 433
Trainable params: 433
Non-trainable params: 0
_________________________________________________________________
None
Accuracy: 89.999998


In [ ]:
test = ['Great ']

In [ ]:
VOCAB_SIZE = 50
encoded_docs_test = [one_hot(d, VOCAB_SIZE) for d in test]

In [ ]:
max_length = 4
padded_docs = pad_sequences(encoded_docs_test, maxlen=max_length, padding='post')

In [ ]:
model.predict(padded_docs)

array([[0.53869087]], dtype=float32)

In [ ]:
from numpy import array
from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
# define documents
docs = ['Well done!',
		'Good work',
		'Great effort',
		'nice work',
		'Excellent!',
		'Weak',
		'Poor effort!',
		'not good',
		'poor work',
		'Could have done better.']
# define class labels
labels = array([1,1,1,1,1,0,0,0,0,0])
# prepare tokenizer
t = Tokenizer()
t.fit_on_texts(docs)
vocab_size = len(t.word_index) + 1 
# integer encode the documents
encoded_docs = t.texts_to_sequences(docs) # Convert text to sequences consistently
print(encoded_docs)
# pad documents to a max length of 4 words
max_length = 4
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs)
# load the whole embedding into memory
embeddings_index = dict()
f = open('../glove_data/glove.6B/glove.6B.100d.txt') # download the pre trained Glove Word Embedding model weights
for line in f:
	values = line.split()
	word = values[0]
	coefs = asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))
# create a weight matrix for words in training docs
embedding_matrix = zeros((vocab_size, 100))
for word, i in t.word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector
# define model
model = Sequential()
e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=4, trainable=False)
model.add(e)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# summarize the model
print(model.summary())
# fit the model
model.fit(padded_docs, labels, epochs=50, verbose=0)
# evaluate the model
loss, accuracy = model.evaluate(padded_docs, labels, verbose=0)
print('Accuracy: %f' % (accuracy*100))